# Chapter 9: Testing and Debugging

---

Good tests are an insurance policy on your code. They give you confidence that your code is correct and make refactoring easier. This chapter covers essential testing and debugging practices in Python.

## Item 75: Use repr Strings for Debugging Output

### The Problem with print

When debugging, `print()` outputs a human-readable string but doesn't always show the actual type and composition of values.

In [ ]:
# Basic printing - type information is hidden
print(5)
print('5')

int_value = 5
str_value = '5'
print(f'{int_value} == {str_value} ?')

### The repr Solution

The `repr()` function returns the **printable representation** of an object, making type differences clear.

In [ ]:
# Using repr to see actual types
print(repr(5))
print(repr('5'))

# Shows the difference clearly
int_value = 5
str_value = '5'
print(f'{int_value!r} != {str_value!r}')

In [ ]:
# repr shows special characters clearly
a = '\x07'  # Bell character
print(repr(a))

# Newlines and tabs are also visible
b = 'Hello\nWorld\tTab'
print('Normal print:', b)
print('With repr:', repr(b))

### Custom __repr__ for Classes

Define `__repr__` to make your objects more debuggable.

In [ ]:
# Without custom __repr__
class OpaqueClass:
    def __init__(self, x, y):
        self.x = x
        self.y = y

obj = OpaqueClass(1, 'foo')
print(obj)  # Not very helpful

In [ ]:
# With custom __repr__
class BetterClass:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f'BetterClass({self.x!r}, {self.y!r})'

obj = BetterClass(2, 'bar')
print(obj)  # Much more informative!

In [ ]:
# Using __dict__ when you can't modify the class
obj = OpaqueClass(4, 'baz')
print(obj.__dict__)

### Practical Examples

In [ ]:
# Example: Debugging a data structure
data = {
    'name': 'Alice',
    'age': 30,
    'city': 'NYC\n',  # Hidden newline!
    'score': '95'     # String, not int!
}

print('Normal print:')
for key, value in data.items():
    print(f'{key}: {value}')

print('\nWith repr:')
for key, value in data.items():
    print(f'{key}: {value!r}')

## Item 76: Verify Related Behaviors in TestCase Subclasses

### Basic Testing with unittest

The `unittest` module provides a framework for writing and running tests.

In [ ]:
# Create a simple function to test
def to_str(data):
    if isinstance(data, str):
        return data
    elif isinstance(data, bytes):
        return data.decode('utf-8')
    else:
        raise TypeError('Must supply str or bytes, '
                       f'found: {data!r}')

In [ ]:
from unittest import TestCase, main

class UtilsTestCase(TestCase):
    def test_to_str_bytes(self):
        self.assertEqual('hello', to_str(b'hello'))
    
    def test_to_str_str(self):
        self.assertEqual('hello', to_str('hello'))
    
    def test_to_str_bad(self):
        with self.assertRaises(TypeError):
            to_str(object())

# Run tests in Jupyter
import sys
suite = unittest.TestLoader().loadTestsFromTestCase(UtilsTestCase)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

### TestCase Helper Methods

TestCase provides many assertion methods that give better error messages than plain `assert`.

In [ ]:
class AssertionExamples(TestCase):
    def test_assert_equal(self):
        """assertEqual shows both values on failure"""
        expected = 12
        found = 2 * 5
        # This will fail and show: AssertionError: 12 != 10
        # self.assertEqual(expected, found)
        pass  # Commented out to avoid test failure
    
    def test_assert_true(self):
        """assertTrue for boolean conditions"""
        self.assertTrue(5 > 3)
        self.assertTrue('hello'.startswith('hel'))
    
    def test_assert_in(self):
        """assertIn for membership testing"""
        self.assertIn('a', 'abc')
        self.assertIn(2, [1, 2, 3])
    
    def test_assert_is_instance(self):
        """assertIsInstance for type checking"""
        self.assertIsInstance(42, int)
        self.assertIsInstance('hello', str)

# Run the examples
suite = unittest.TestLoader().loadTestsFromTestCase(AssertionExamples)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

### Custom Test Helpers

In [ ]:
def sum_squares(values):
    """Generate cumulative sum of squares"""
    cumulative = 0
    for value in values:
        cumulative += value ** 2
        yield cumulative

class HelperTestCase(TestCase):
    def verify_sum_squares(self, values, expected):
        """Custom helper for testing generators"""
        expect_it = iter(expected)
        found_it = iter(sum_squares(values))
        test_it = zip(expect_it, found_it)
        
        for i, (expect, found) in enumerate(test_it):
            self.assertEqual(
                expect,
                found,
                f'Index {i} is wrong')
        
        # Verify both generators are exhausted
        try:
            next(expect_it)
        except StopIteration:
            pass
        else:
            self.fail('Expected longer than found')
        
        try:
            next(found_it)
        except StopIteration:
            pass
        else:
            self.fail('Found longer than expected')
    
    def test_correct_case(self):
        values = [1, 2, 3]
        expected = [
            1**2,
            1**2 + 2**2,
            1**2 + 2**2 + 3**2
        ]
        self.verify_sum_squares(values, expected)

suite = unittest.TestLoader().loadTestsFromTestCase(HelperTestCase)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

### Data-Driven Tests with subTest

In [ ]:
class DataDrivenTestCase(TestCase):
    def test_multiple_cases(self):
        """Test multiple cases without stopping at first failure"""
        test_cases = [
            (b'my bytes', 'my bytes'),
            ('string', 'string'),
            (b'utf-8 bytes', 'utf-8 bytes'),
        ]
        
        for value, expected in test_cases:
            with self.subTest(value=value):
                self.assertEqual(expected, to_str(value))
    
    def test_error_cases(self):
        """Test multiple error conditions"""
        error_cases = [
            (object(), TypeError),
            (123, TypeError),
            (None, TypeError),
        ]
        
        for value, exception in error_cases:
            with self.subTest(value=value):
                with self.assertRaises(exception):
                    to_str(value)

suite = unittest.TestLoader().loadTestsFromTestCase(DataDrivenTestCase)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

## Item 77: Isolate Tests with setUp, tearDown, setUpModule, and tearDownModule

### Test Isolation with setUp and tearDown

These methods run before and after each test method, ensuring test isolation.

In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

class EnvironmentTest(TestCase):
    def setUp(self):
        """Create clean environment before each test"""
        self.test_dir = TemporaryDirectory()
        self.test_path = Path(self.test_dir.name)
        print(f'\nSetUp: Created {self.test_path}')
    
    def tearDown(self):
        """Clean up after each test"""
        print(f'TearDown: Cleaning {self.test_path}')
        self.test_dir.cleanup()
    
    def test_create_file(self):
        """Test file creation"""
        file_path = self.test_path / 'test.txt'
        file_path.write_text('test data')
        self.assertTrue(file_path.exists())
        print(f'Test: Created {file_path}')
    
    def test_create_directory(self):
        """Test directory creation"""
        dir_path = self.test_path / 'subdir'
        dir_path.mkdir()
        self.assertTrue(dir_path.exists())
        print(f'Test: Created {dir_path}')

suite = unittest.TestLoader().loadTestsFromTestCase(EnvironmentTest)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

### Module-Level Setup and Teardown

For expensive setup operations, use module-level functions.

In [ ]:
# Example of module-level setup (would be in a separate module)
# This demonstrates the concept

# def setUpModule():
#     print('* Module setup - expensive operation')
#     # Start database, load data, etc.
# 
# def tearDownModule():
#     print('* Module teardown - cleanup')
#     # Stop database, cleanup resources, etc.

class IntegrationTest(TestCase):
    @classmethod
    def setUpClass(cls):
        """Alternative: class-level setup"""
        print('\n* Class setup')
    
    @classmethod
    def tearDownClass(cls):
        print('* Class teardown')
    
    def test_one(self):
        print('  Test 1')
        self.assertTrue(True)
    
    def test_two(self):
        print('  Test 2')
        self.assertTrue(True)

suite = unittest.TestLoader().loadTestsFromTestCase(IntegrationTest)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

## Item 78: Use Mocks to Test Code with Complex Dependencies

### Introduction to Mocking

Mocks simulate dependencies that are difficult or slow to set up.

In [ ]:
from unittest.mock import Mock
from datetime import datetime

# Create a mock function
def get_animals(database, species):
    """Would normally query a database"""
    pass

# Create a mock
mock = Mock(spec=get_animals)
expected = [
    ('Spot', datetime(2019, 6, 5, 11, 15)),
    ('Fluffy', datetime(2019, 6, 5, 12, 30)),
    ('Jojo', datetime(2019, 6, 5, 12, 45)),
]
mock.return_value = expected

# Use the mock
database = object()  # Sentinel value
result = mock(database, 'Meerkat')

print('Returned:', result)
print('Same as expected:', result == expected)

# Verify the mock was called correctly
mock.assert_called_once_with(database, 'Meerkat')

### Using ANY for Flexible Assertions

In [ ]:
from unittest.mock import ANY

mock = Mock(spec=get_animals)
mock('database 1', 'Rabbit')
mock('database 2', 'Bison')
mock('database 3', 'Meerkat')

# Verify last call with ANY for database parameter
mock.assert_called_with(ANY, 'Meerkat')
print('Mock assertions passed!')

### Mocking Exceptions

In [ ]:
class DatabaseError(Exception):
    pass

mock = Mock(spec=get_animals)
mock.side_effect = DatabaseError('Connection failed')

try:
    mock(database, 'Meerkat')
except DatabaseError as e:
    print(f'Caught expected error: {e}')

### Complete Mock Testing Example

In [ ]:
from datetime import timedelta
from unittest.mock import call

# Function to test
def do_rounds(database, species, *, utcnow=datetime.utcnow):
    """Feed animals that need feeding"""
    now = utcnow()
    feeding_timedelta = database.get_food_period(species)
    animals = database.get_animals(species)
    fed = 0
    
    for name, last_mealtime in animals:
        if (now - last_mealtime) >= feeding_timedelta:
            database.feed_animal(name, now)
            fed += 1
    
    return fed

# Mock the database
class ZooDatabase:
    def get_animals(self, species): pass
    def get_food_period(self, species): pass
    def feed_animal(self, name, when): pass

# Create test
database = Mock(spec=ZooDatabase)
now_func = Mock(spec=datetime.utcnow)
now_func.return_value = datetime(2019, 6, 5, 15, 45)

database.get_food_period.return_value = timedelta(hours=3)
database.get_animals.return_value = [
    ('Spot', datetime(2019, 6, 5, 11, 15)),   # Needs food
    ('Fluffy', datetime(2019, 6, 5, 12, 30)), # Needs food
    ('Jojo', datetime(2019, 6, 5, 12, 55))    # Recent, doesn't need
]

# Run the test
result = do_rounds(database, 'Meerkat', utcnow=now_func)

print(f'Fed {result} animals')

# Verify calls
database.get_food_period.assert_called_once_with('Meerkat')
database.get_animals.assert_called_once_with('Meerkat')
database.feed_animal.assert_has_calls(
    [
        call('Spot', now_func.return_value),
        call('Fluffy', now_func.return_value),
    ],
    any_order=True
)

print('All assertions passed!')

## Item 79: Encapsulate Dependencies to Facilitate Mocking

### Better Design for Testing

Encapsulating dependencies in classes makes testing easier.

In [ ]:
# Bad: Functions as dependencies (harder to mock)
# def get_animals(database, species): ...
# def get_food_period(database, species): ...
# def feed_animal(database, name, when): ...

# Good: Class encapsulation (easier to mock)
class ZooDatabase:
    def __init__(self, connection_string):
        self.connection_string = connection_string
    
    def get_animals(self, species):
        """Query database for animals"""
        pass
    
    def get_food_period(self, species):
        """Get feeding interval"""
        pass
    
    def feed_animal(self, name, when):
        """Record feeding time"""
        pass

# Now mocking is straightforward
database_mock = Mock(spec=ZooDatabase)
print('Created mock:', database_mock)
print('Mock method:', database_mock.feed_animal)

### Dependency Injection Pattern

In [ ]:
# Global database instance
DATABASE = None

def get_database():
    """Get or create database instance"""
    global DATABASE
    if DATABASE is None:
        DATABASE = ZooDatabase('connection_string')
    return DATABASE

def main_program(species):
    """Main program using dependency injection"""
    database = get_database()
    count = do_rounds(database, species)
    return count

# For testing, we can easily inject a mock
mock_db = Mock(spec=ZooDatabase)
mock_db.get_food_period.return_value = timedelta(hours=3)
mock_db.get_animals.return_value = [
    ('Test', datetime(2019, 6, 5, 11, 15))
]

# Inject the mock
result = do_rounds(mock_db, 'Meerkat')
print(f'Test result: fed {result} animals')

## Item 80: Consider Interactive Debugging with pdb

### Using breakpoint()

The `breakpoint()` function starts the debugger.

In [ ]:
import math

def compute_rmse(observed, ideal):
    """Compute Root Mean Square Error"""
    total_err_2 = 0
    count = 0
    
    for got, wanted in zip(observed, ideal):
        err_2 = (got - wanted) ** 2
        # breakpoint()  # Uncomment to start debugger
        total_err_2 += err_2
        count += 1
    
    mean_err = total_err_2 / count
    rmse = math.sqrt(mean_err)
    return rmse

# Test the function
result = compute_rmse(
    [1.8, 1.7, 3.2, 6],
    [2, 1.5, 3, 5]
)
print(f'RMSE: {result:.2f}')

### Conditional Breakpoints

In [ ]:
def compute_rmse_conditional(observed, ideal):
    """RMSE with conditional debugging"""
    total_err_2 = 0
    count = 0
    
    for got, wanted in zip(observed, ideal):
        err_2 = (got - wanted) ** 2
        
        # Only debug when error is large
        if err_2 >= 1:
            # breakpoint()  # Uncomment to debug large errors
            print(f'Large error detected: {err_2}')
        
        total_err_2 += err_2
        count += 1
    
    mean_err = total_err_2 / count
    rmse = math.sqrt(mean_err)
    return rmse

result = compute_rmse_conditional(
    [1.8, 1.7, 3.2, 7],
    [2, 1.5, 3, 5]
)
print(f'RMSE: {result:.2f}')

### Common pdb Commands

When in the debugger:

**Inspection:**
- `p <expr>` - Print expression
- `pp <expr>` - Pretty-print expression
- `where` - Show call stack
- `up` - Move up call stack
- `down` - Move down call stack
- `locals()` - Show local variables

**Execution:**
- `step` / `s` - Step into function
- `next` / `n` - Next line (skip function)
- `return` / `r` - Return from function
- `continue` / `c` - Continue execution
- `quit` / `q` - Exit debugger

## Item 81: Use tracemalloc to Understand Memory Usage

### Memory Debugging with gc

In [ ]:
import gc
import os

class MyObject:
    def __init__(self):
        self.data = os.urandom(100)

def create_objects():
    values = []
    for _ in range(100):
        obj = MyObject()
        values.append(obj)
    return values

# Check object count
found_objects = gc.get_objects()
print(f'Before: {len(found_objects)} objects')

# Create objects
hold_reference = create_objects()

found_objects = gc.get_objects()
print(f'After: {len(found_objects)} objects')
print(f'\nSample objects:')
for obj in found_objects[:3]:
    print(f'  {repr(obj)[:60]}')

### Better Memory Debugging with tracemalloc

In [ ]:
import tracemalloc

# Start tracing
tracemalloc.start(10)  # Keep 10 stack frames

# Take before snapshot
time1 = tracemalloc.take_snapshot()

# Create objects
x = create_objects()

# Take after snapshot
time2 = tracemalloc.take_snapshot()

# Compare snapshots
stats = time2.compare_to(time1, 'lineno')

print('Top 3 memory allocations:')
for stat in stats[:3]:
    print(stat)

tracemalloc.stop()

### Detailed Stack Traces

In [ ]:
tracemalloc.start(10)

time1 = tracemalloc.take_snapshot()
x = create_objects()
time2 = tracemalloc.take_snapshot()

# Get statistics by traceback
stats = time2.compare_to(time1, 'traceback')
top = stats[0]

print('Biggest memory offender:')
print('\n'.join(top.traceback.format()))

tracemalloc.stop()

### Practical Memory Leak Detection

In [ ]:
# Simulate a memory leak
leaked_objects = []

def potentially_leaky_function():
    """This function 'leaks' by keeping references"""
    data = [MyObject() for _ in range(50)]
    leaked_objects.extend(data)  # Oops, kept reference!
    return len(data)

tracemalloc.start()

snapshots = []
for i in range(3):
    potentially_leaky_function()
    snapshot = tracemalloc.take_snapshot()
    snapshots.append(snapshot)

# Compare first and last snapshot
stats = snapshots[-1].compare_to(snapshots[0], 'lineno')

print('Memory growth:')
for stat in stats[:3]:
    print(stat)

tracemalloc.stop()

# Clean up
leaked_objects.clear()

## Summary: Testing and Debugging Best Practices

### Key Takeaways

1. **Use repr() for debugging** - Makes types and values clear
2. **Write comprehensive tests** - Use unittest.TestCase for organized testing
3. **Isolate tests** - Use setUp/tearDown for clean test environments
4. **Mock dependencies** - Use unittest.mock for complex dependencies
5. **Design for testability** - Encapsulate dependencies in classes
6. **Interactive debugging** - Use breakpoint() when tests aren't enough
7. **Monitor memory** - Use tracemalloc to find memory issues

### Testing Hierarchy

```
Unit Tests (fast, isolated)
    ↓
Integration Tests (realistic, slower)
    ↓
End-to-End Tests (complete system)
```

### When to Use Each Tool

- **unittest**: Standard testing framework
- **Mock**: Complex dependencies, slow operations
- **pdb**: Interactive debugging, unclear failures
- **tracemalloc**: Memory leaks, performance issues

## Practice Exercises

Try these exercises to reinforce your learning:

In [ ]:
# Exercise 1: Write tests for this function
def calculate_average(numbers):
    """Calculate the average of a list of numbers"""
    if not numbers:
        raise ValueError('Cannot calculate average of empty list')
    return sum(numbers) / len(numbers)

# TODO: Write TestCase with multiple test methods
# - Test with normal input
# - Test with empty list
# - Test with single number
# - Test with negative numbers

In [ ]:
# Exercise 2: Create a mock for this class
class EmailService:
    def send_email(self, to, subject, body):
        """Send an email (would normally contact SMTP server)"""
        pass
    
    def get_inbox(self, user):
        """Get user's inbox (would normally query server)"""
        pass

def notify_user(email_service, user, message):
    """Send notification to user"""
    email_service.send_email(
        to=user,
        subject='Notification',
        body=message
    )

# TODO: Create a mock for EmailService
# TODO: Test notify_user with the mock
# TODO: Verify send_email was called correctly

In [ ]:
# Exercise 3: Debug this function
def find_duplicates(items):
    """Find duplicate items in a list"""
    seen = set()
    duplicates = []
    
    for item in items:
        if item in seen:
            duplicates.append(item)
        seen.add(item)
    
    return duplicates

# Test it
result = find_duplicates([1, 2, 3, 2, 4, 3, 5])
print('Duplicates:', result)

# TODO: What happens with multiple duplicates?
# TODO: Add repr() debugging to understand the issue
# TODO: Fix the function if needed